# Heatmap / df_points with UNIX Timestamp #
# Creates df_points.csv with all points and idletimes #

### ca 1h for 1 year

In [230]:
def importdata():

    # show all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # import Data
    df01 = pd.read_csv('/Users/jonweske/devment/BA_nextbike/data/lendings_2019.csv')


    df01 = df01[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

    # cut
    #df01 = df01.truncate(after=10000)
    return df01

In [231]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])

    return myDFs

In [232]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [233]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [234]:
def dfPoints(myDFs):
    # DF of all points with stoodtime
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'timestood', 'time_start', 'time_end'])
    df_points[['bike_id', 'timestood', 'time_start', 'time_end']] = df_points[['bike_id', 'timestood', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            timestood = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'timestood': timestood, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            df_points = df_points.append(dict, ignore_index=True)
    print(inva)
    return df_points

In [235]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main

In [236]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from geopy.distance import geodesic 
df = pd.DataFrame()


df = importdata()
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)


# create DF_POints
#df_points = dfPoints(myDfs)


# save CSV and Map
#df_points.to_csv('df_points.csv', index=False)
#keplerMap(df_points)


  0%|▋                                                                                                                                                                                                 | 1989/604598 [00:02<14:24, 697.37it/s]


KeyboardInterrupt: 

## Save MyDfs Map to file 

In [ ]:
import pickle
with open('saved_myDfs.pkl', 'wb') as f:
    pickle.dump(myDfs, f)